In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
%%writefile MRJobGraph1.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re, sys

# This MrJob calculates the gradient of the entire training set 
#     Mapper: calculate partial gradient for each example  
#     
class MRJobGraph1(MRJob):
    # run before the mapper processes any input
    def label_dataset(self, _, line):
        
        source_node_id = 1
        # Read dataset and add labels and distances
        splits = line.split("\t")
        node_id = int(splits[0])
        edges = eval(splits[1])
        if (node_id == source_node_id):
            yield None, str(node_id) + "\t" + str([int(x) for x in edges.keys()]) + "|0|Q"
        else:
            yield None, str(node_id) + "\t" + str([int(x) for x in edges.keys()]) + "|"+str(sys.maxint)+"|U"
    

    
    def steps(self):
        return [MRStep(mapper=self.label_dataset)]
    
if __name__ == '__main__':
    MRJobGraph1.run()

Overwriting MRJobGraph1.py


In [26]:
from numpy import random,array
from MRJobGraph1 import MRJobGraph1


# create a mrjob instance for batch gradient descent update over all data
mr_job = MRJobGraph1(args=['undirected_toy.txt', '--no-strict-protocol'])
with mr_job.make_runner() as runner: 
    runner.run()
    #no output yet
    for line in runner.stream_output():
        key, value = mr_job.parse_output_line(line)
        print value


1	[2, 5]|0|Q
2	[1, 3, 5, 4]|9223372036854775807|U
3	[2, 4]|9223372036854775807|U
4	[3, 2, 5]|9223372036854775807|U
5	[1, 2, 4]|9223372036854775807|U


In [2]:
testDict = {'1': 1,'3': 1,'4': 1,'5': 1}

In [4]:
[int(x) for x in testDict.keys()]

[1, 3, 5, 4]

In [8]:
print str(7) + "\t" + str([int(x) for x in testDict.keys()]) + "|0|Q"

7	[1, 3, 5, 4]|0|Q
